In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
df =  pd.read_csv("results.csv")

In [3]:
df["date"] = pd.to_datetime(df["date"])

In [4]:
df.isna().sum()

date          0
home_team     0
away_team     0
home_score    0
away_score    0
tournament    0
city          0
country       0
neutral       0
dtype: int64

In [5]:
df.sort_values("date").tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
44336,2022-12-04,England,Senegal,3,0,FIFA World Cup,Al Khor,Qatar,True
44337,2022-12-05,Japan,Croatia,1,1,FIFA World Cup,Al Wakrah,Qatar,True
44338,2022-12-05,Brazil,South Korea,4,1,FIFA World Cup,Doha,Qatar,True
44339,2022-12-06,Morocco,Spain,0,0,FIFA World Cup,Al Rayyan,Qatar,True
44340,2022-12-06,Portugal,Switzerland,6,1,FIFA World Cup,Lusail,Qatar,True


In [6]:
df = df[(df["date"] >= "2010-8-1")].reset_index(drop=True)

In [8]:
rankings = pd.read_csv("fifa_ranking-2022-10-06.csv")
rankings.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.0,0.0,0,UEFA,1992-12-31
1,96,Syria,SYR,11.0,0.0,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.0,0.0,0,CAF,1992-12-31
3,99,Latvia,LVA,10.0,0.0,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.0,0.0,0,CAF,1992-12-31


In [9]:
sum(rankings['country_full']=='Argentina')

322

In [10]:
rankings["rank_date"] = pd.to_datetime(rankings["rank_date"])
rankings = rankings[(rankings["rank_date"] >= "2006-8-1")].reset_index(drop=True)

In [11]:
rankings["country_full"] = rankings["country_full"].str.replace("IR Iran", "Iran").str.replace("Korea Republic", "South Korea").str.replace("USA", "United States")

In [12]:
df_wc_ranked = df.merge(rankings[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "home_team"], right_on=["rank_date", "country_full"]).drop(["rank_date", "country_full"], axis=1)
df_wc_ranked.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points,previous_points,rank,rank_change
0,2010-08-11,Albania,Uzbekistan,1,0,Friendly,Durrës,Albania,False,455.0,455.0,70,-1
1,2010-08-11,Algeria,Gabon,1,2,Friendly,Algiers,Algeria,False,759.0,759.0,33,0
2,2010-08-11,Angola,Uruguay,0,2,Friendly,Lisbon,Portugal,True,357.0,356.0,87,1
3,2010-08-11,Armenia,Iran,1,3,Friendly,Yerevan,Armenia,False,318.0,318.0,96,0
4,2010-08-11,Austria,Switzerland,0,1,Friendly,Klagenfurt,Austria,False,536.0,536.0,60,0


In [13]:
df_wc_ranked = df_wc_ranked.merge(rankings[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "away_team"], right_on=["rank_date", "country_full"], suffixes=("_home", "_away")).drop(["rank_date", "country_full"], axis=1)
df_wc_ranked.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points_home,previous_points_home,rank_home,rank_change_home,total_points_away,previous_points_away,rank_away,rank_change_away
0,2010-08-11,Albania,Uzbekistan,1,0,Friendly,Durrës,Albania,False,455.0,455.0,70,-1,351.0,352.0,88,1
1,2010-08-11,Algeria,Gabon,1,2,Friendly,Algiers,Algeria,False,759.0,759.0,33,0,755.0,755.0,34,0
2,2010-08-11,Angola,Uruguay,0,2,Friendly,Lisbon,Portugal,True,357.0,356.0,87,1,1152.0,1152.0,6,0
3,2010-08-11,Armenia,Iran,1,3,Friendly,Yerevan,Armenia,False,318.0,318.0,96,0,517.0,522.0,65,1
4,2010-08-11,Austria,Switzerland,0,1,Friendly,Klagenfurt,Austria,False,536.0,536.0,60,0,940.0,940.0,17,-1


In [14]:
df = df_wc_ranked

In [15]:
def result_finder(home, away):
    if home > away:
        return pd.Series([0, 3, 0])
    if home < away:
        return pd.Series([1, 0, 3])
    else:
        return pd.Series([2, 1, 1])

results = df.apply(lambda x: result_finder(x["home_score"], x["away_score"]), axis=1)
df[["result", "home_team_points", "away_team_points"]] = results


In [16]:
df_wc_ranked.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points_home,previous_points_home,rank_home,rank_change_home,total_points_away,previous_points_away,rank_away,rank_change_away,result,home_team_points,away_team_points
0,2010-08-11,Albania,Uzbekistan,1,0,Friendly,Durrës,Albania,False,455.0,455.0,70,-1,351.0,352.0,88,1,0,3,0
1,2010-08-11,Algeria,Gabon,1,2,Friendly,Algiers,Algeria,False,759.0,759.0,33,0,755.0,755.0,34,0,1,0,3
2,2010-08-11,Angola,Uruguay,0,2,Friendly,Lisbon,Portugal,True,357.0,356.0,87,1,1152.0,1152.0,6,0,1,0,3
3,2010-08-11,Armenia,Iran,1,3,Friendly,Yerevan,Armenia,False,318.0,318.0,96,0,517.0,522.0,65,1,1,0,3
4,2010-08-11,Austria,Switzerland,0,1,Friendly,Klagenfurt,Austria,False,536.0,536.0,60,0,940.0,940.0,17,-1,1,0,3


In [17]:
df["rank_dif"] = df["rank_home"] - df["rank_away"]
df["sg"] = df["home_score"] - df["away_score"]
df["points_home_by_rank"] = df["home_team_points"]/df["rank_away"]
df["points_away_by_rank"] = df["away_team_points"]/df["rank_home"]


In [18]:
df["rank_dif"] = df["rank_home"] - df["rank_away"]
df["sg"] = df["home_score"] - df["away_score"]
df["points_home_by_rank"] = df["home_team_points"]/df["rank_away"]
df["points_away_by_rank"] = df["away_team_points"]/df["rank_home"]

In [19]:
home_team = df[["date", "home_team", "home_score", "away_score", "rank_home", "rank_away","rank_change_home", "total_points_home", "result", "rank_dif", "points_home_by_rank", "home_team_points"]]

away_team = df[["date", "away_team", "away_score", "home_score", "rank_away", "rank_home","rank_change_away", "total_points_away", "result", "rank_dif", "points_away_by_rank", "away_team_points"]]

In [20]:
home_team.columns = [h.replace("home_", "").replace("_home", "").replace("away_", "suf_").replace("_away", "_suf") for h in home_team.columns]

away_team.columns = [a.replace("away_", "").replace("_away", "").replace("home_", "suf_").replace("_home", "_suf") for a in away_team.columns]

In [21]:
team_stats = home_team.append(away_team)

C:\Users\USER\AppData\Local\Temp\ipykernel_2524\2656291173.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  team_stats = home_team.append(away_team)


In [22]:
team_stats_raw = team_stats.copy()
team_stats_raw .head()

,date,team,score,suf_score,rank,rank_suf,rank_change,total_points,result,rank_dif,points_by_rank,team_points
0,2010-08-11,Albania,1,0,70,88,-1,455.0,0,-18,0.034091,3
1,2010-08-11,Algeria,1,2,33,34,0,759.0,1,-1,0.000000,0
2,2010-08-11,Angola,0,2,87,6,1,357.0,1,81,0.000000,0
3,2010-08-11,Armenia,1,3,96,65,0,318.0,1,31,0.000000,0
4,2010-08-11,Austria,0,1,60,17,0,536.0,1,43,0.000000,0


In [23]:
from random import randrange
stats_val = []
import numpy as np
for index, row in team_stats.iterrows():
    team = row["team"]
    date = row["date"]
    past_games = team_stats.loc[(team_stats["team"] == team) & (team_stats["date"] < date)].sort_values(by=['date'], ascending=False)
    last5 = past_games.head()
    if np.isnan(past_games["score"].mean())==False :
            goals_l5 = last5["score"].mean()
            goals = past_games["score"].mean()
            
    else: 
            goals = randrange(0,3,1)
            goals_l5 = randrange(0,3,1)
    if (np.isnan(past_games["suf_score"].mean())==False) and (np.isnan(past_games.head(5).mean==False)):
            goals_suf_l5 = last5["suf_score"].mean()
            goals_suf = past_games["suf_score"].mean()
           
    else : 
            goals_suf = randrange(0,3,1)
            goals_suf_l5 = randrange(0,3,1)
    if np.isnan(past_games["rank_suf"].mean()) ==False:
            rank = past_games["rank_suf"].mean()
            rank_l5 = last5["rank_suf"].mean()
    else : 
            rank =randrange(5,45,1)
            rank_l5 = randrange(5,45,1)
    
    if len(last5) > 0:
        points = past_games["total_points"].values[0] - past_games["total_points"].values[-1]
        points_l5 = last5["total_points"].values[0] - last5["total_points"].values[-1] 
    else:
        points = 0
        points_l5 = 0
    if np.isnan(past_games["team_points"].mean())==False:
            gp = past_games["team_points"].mean()
            gp_l5 = last5["team_points"].mean()
    else : 
            gp = randrange(0,3,1)
            gp_l5 = randrange(0,3,1)
    if np.isnan(past_games["points_by_rank"].mean())==False:
        
            gp_rank = past_games["points_by_rank"].mean()
            gp_rank_l5 = last5["points_by_rank"].mean()
    else :
            gp_rank = randrange(0,1,1)
            gp_rank_l5 = randrange(0,1,1)
    
    stats_val.append([goals, goals_l5, goals_suf,goals_suf_l5, rank, rank_l5, points, points_l5, gp, gp_l5, gp_rank, gp_rank_l5])


In [24]:
stats_cols = ["goals_mean", "goals_mean_l5", "goals_suf_mean","goals_suf_mean_l5", "rank_mean", "rank_mean_l5", "points_mean", "points_mean_l5", "game_points_mean", "game_points_mean_l5", "game_points_rank_mean", "game_points_rank_mean_l5"]

stats_df = pd.DataFrame(stats_val, columns=stats_cols)

full_df = pd.concat([team_stats.reset_index(drop=True), stats_df], axis=1, ignore_index=False)
full_df

,date,team,score,suf_score,rank,rank_suf,rank_change,total_points,result,rank_dif,...,goals_suf_mean,goals_suf_mean_l5,rank_mean,rank_mean_l5,points_mean,points_mean_l5,game_points_mean,game_points_mean_l5,game_points_rank_mean,game_points_rank_mean_l5
0,2010-08-11,Albania,1,0,70,88,-1,455.00,0,-18,...,0,1,18.00,40.00,0.0,0.0,1.00,2.00,0.000000,0.000000
1,2010-08-11,Algeria,1,2,33,34,0,759.00,1,-1,...,1,1,32.00,14.00,0.0,0.0,2.00,0.00,0.000000,0.000000
2,2010-08-11,Angola,0,2,87,6,1,357.00,1,81,...,2,0,34.00,41.00,0.0,0.0,1.00,1.00,0.000000,0.000000
3,2010-08-11,Armenia,1,3,96,65,0,318.00,1,31,...,2,1,35.00,16.00,0.0,0.0,2.00,2.00,0.000000,0.000000
4,2010-08-11,Austria,0,1,60,17,0,536.00,1,43,...,1,2,31.00,17.00,0.0,0.0,2.00,2.00,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,2019-06-14,Mali,1,1,62,51,-3,1365.00,2,-11,...,0,1,102.00,102.00,0.0,0.0,0.00,0.00,0.000000,0.000000
370,2019-06-14,Mauritania,3,1,103,108,0,1210.00,1,5,...,0,1,171.00,171.00,0.0,0.0,1.00,1.00,0.005848,0.005848
371,2019-06-14,Bolivia,0,3,62,3,-1,1365.00,0,-59,...,2,0,46.50,46.50,-41.0,-41.0,1.00,1.00,0.022661,0.022661
372,2021-05-27,Azerbaijan,1,2,110,29,0,1168.53,0,-81,...,2,0,52.25,52.25,114.0,114.0,1.25,1.25,0.029727,0.029727


In [25]:
full_df.isna().sum()

date                        0
team                        0
score                       0
suf_score                   0
rank                        0
rank_suf                    0
rank_change                 0
total_points                0
result                      0
rank_dif                    0
points_by_rank              0
team_points                 0
goals_mean                  0
goals_mean_l5               0
goals_suf_mean              0
goals_suf_mean_l5           0
rank_mean                   0
rank_mean_l5                0
points_mean                 0
points_mean_l5              0
game_points_mean            0
game_points_mean_l5         0
game_points_rank_mean       0
game_points_rank_mean_l5    0
dtype: int64

In [26]:
home_team_stats = full_df.iloc[:int(full_df.shape[0]/2),:]
away_team_stats = full_df.iloc[int(full_df.shape[0]/2):,:]

In [27]:
home_team_stats = home_team_stats[home_team_stats.columns[-12:]]
away_team_stats = away_team_stats[away_team_stats.columns[-12:]]

In [28]:
home_team_stats.columns = ['home_'+str(col) for col in home_team_stats.columns]
away_team_stats.columns = ['away_'+str(col) for col in away_team_stats.columns]

In [29]:
match_stats = pd.concat([home_team_stats, away_team_stats.reset_index(drop=True)], axis=1, ignore_index=False)
full_df = pd.concat([df, match_stats.reset_index(drop=True)], axis=1, ignore_index=False)
full_df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points_home,...,away_goals_suf_mean,away_goals_suf_mean_l5,away_rank_mean,away_rank_mean_l5,away_points_mean,away_points_mean_l5,away_game_points_mean,away_game_points_mean_l5,away_game_points_rank_mean,away_game_points_rank_mean_l5
0,2010-08-11,Albania,Uzbekistan,1,0,Friendly,Durrës,Albania,False,455.0,...,1,2,9.0,28.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2010-08-11,Algeria,Gabon,1,2,Friendly,Algiers,Algeria,False,759.0,...,1,2,35.0,21.0,0.0,0.0,1.0,2.0,0.0,0.0
2,2010-08-11,Angola,Uruguay,0,2,Friendly,Lisbon,Portugal,True,357.0,...,1,2,19.0,20.0,0.0,0.0,1.0,2.0,0.0,0.0
3,2010-08-11,Armenia,Iran,1,3,Friendly,Yerevan,Armenia,False,318.0,...,2,1,7.0,21.0,0.0,0.0,2.0,1.0,0.0,0.0
4,2010-08-11,Austria,Switzerland,0,1,Friendly,Klagenfurt,Austria,False,536.0,...,2,1,27.0,30.0,0.0,0.0,2.0,0.0,0.0,0.0


In [30]:
def find_friendly(x):
    if x == "Friendly":
        return 1
    else: return 0

full_df["is_friendly"] = full_df["tournament"].apply(lambda x: find_friendly(x))
full_df = pd.get_dummies(full_df, columns=["is_friendly"])

In [31]:
base_df = full_df[["date", "home_team", "away_team", "rank_home", "rank_away","home_score", "away_score","result", "rank_dif", "rank_change_home", "rank_change_away", 'home_goals_mean',
       'home_goals_mean_l5', 'home_goals_suf_mean', 'home_goals_suf_mean_l5',
       'home_rank_mean', 'home_rank_mean_l5', 'home_points_mean',
       'home_points_mean_l5', 'away_goals_mean', 'away_goals_mean_l5',
       'away_goals_suf_mean', 'away_goals_suf_mean_l5', 'away_rank_mean',
       'away_rank_mean_l5', 'away_points_mean', 'away_points_mean_l5','home_game_points_mean', 'home_game_points_mean_l5','home_game_points_rank_mean', 'home_game_points_rank_mean_l5','away_game_points_mean',
       'away_game_points_mean_l5', 'away_game_points_rank_mean',
       'away_game_points_rank_mean_l5',
       'is_friendly_0', 'is_friendly_1']]

base_df.tail()

,date,home_team,away_team,rank_home,rank_away,home_score,away_score,result,rank_dif,rank_change_home,...,home_game_points_mean,home_game_points_mean_l5,home_game_points_rank_mean,home_game_points_rank_mean_l5,away_game_points_mean,away_game_points_mean_l5,away_game_points_rank_mean,away_game_points_rank_mean_l5,is_friendly_0,is_friendly_1
182,2019-06-14,Cameroon,Mali,51,62,1,1,2,-11,-3,...,3.000000,3.000000,0.053571,0.053571,0.00,0.00,0.000000,0.000000,0,1
183,2019-06-14,Madagascar,Mauritania,108,103,1,3,1,5,1,...,1.000000,1.000000,0.000000,0.000000,1.00,1.00,0.005848,0.005848,0,1
184,2019-06-14,Brazil,Bolivia,3,62,3,0,0,-59,0,...,0.000000,0.000000,0.000000,0.000000,1.00,1.00,0.022661,0.022661,1,0
185,2021-05-27,Turkey,Azerbaijan,29,110,2,1,0,-81,0,...,1.333333,1.333333,0.027513,0.027513,1.25,1.25,0.029727,0.029727,0,1
186,2021-11-19,Sri Lanka,Seychelles,204,197,3,3,2,7,0,...,1.000000,1.000000,0.006289,0.006289,0.00,0.00,0.000000,0.000000,1,0


In [32]:
df = base_df.copy()

In [33]:
def no_draw(x):
    if x == 2:
        return 1
    else:
        return x
    
df["target"] = df["result"].apply(lambda x: no_draw(x))
df.head()

,date,home_team,away_team,rank_home,rank_away,home_score,away_score,result,rank_dif,rank_change_home,...,home_game_points_mean_l5,home_game_points_rank_mean,home_game_points_rank_mean_l5,away_game_points_mean,away_game_points_mean_l5,away_game_points_rank_mean,away_game_points_rank_mean_l5,is_friendly_0,is_friendly_1,target
0,2010-08-11,Albania,Uzbekistan,70,88,1,0,0,-18,-1,...,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1,0
1,2010-08-11,Algeria,Gabon,33,34,1,2,1,-1,0,...,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0,1,1
2,2010-08-11,Angola,Uruguay,87,6,0,2,1,81,1,...,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0,1,1
3,2010-08-11,Armenia,Iran,96,65,1,3,1,31,0,...,2.0,0.0,0.0,2.0,1.0,0.0,0.0,0,1,1
4,2010-08-11,Austria,Switzerland,60,17,0,1,1,43,0,...,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0,1,1


In [34]:
dif = df.copy()
dif.loc[:, "goals_dif"] = dif["home_goals_mean"] - dif["away_goals_mean"]
dif.loc[:, "goals_dif_l5"] = dif["home_goals_mean_l5"] - dif["away_goals_mean_l5"]
dif.loc[:, "goals_suf_dif"] = dif["home_goals_suf_mean"] - dif["away_goals_suf_mean"]
dif.loc[:, "goals_suf_dif_l5"] = dif["home_goals_suf_mean_l5"] - dif["away_goals_suf_mean_l5"]
dif.loc[:, "goals_made_suf_dif"] = dif["home_goals_mean"] - dif["away_goals_suf_mean"]
dif.loc[:, "goals_made_suf_dif_l5"] = dif["home_goals_mean_l5"] - dif["away_goals_suf_mean_l5"]
dif.loc[:, "goals_suf_made_dif"] = dif["home_goals_suf_mean"] - dif["away_goals_mean"]
dif.loc[:, "goals_suf_made_dif_l5"] = dif["home_goals_suf_mean_l5"] - dif["away_goals_mean_l5"]
dif.loc[:, "dif_points"] = dif["home_game_points_mean"] - dif["away_game_points_mean"]
dif.loc[:, "dif_points_l5"] = dif["home_game_points_mean_l5"] - dif["away_game_points_mean_l5"]
dif.loc[:, "dif_points_rank"] = dif["home_game_points_rank_mean"] - dif["away_game_points_rank_mean"]
dif.loc[:, "dif_points_rank_l5"] = dif["home_game_points_rank_mean_l5"] - dif["away_game_points_rank_mean_l5"]
dif.loc[:, "dif_rank_agst"] = dif["home_rank_mean"] - dif["away_rank_mean"]
dif.loc[:, "dif_rank_agst_l5"] = dif["home_rank_mean_l5"] - dif["away_rank_mean_l5"]
dif.loc[:, "goals_per_ranking_dif"] = (dif["home_goals_mean"] / dif["home_rank_mean"]) - (dif["away_goals_mean"] / dif["away_rank_mean"])
dif.loc[:, "goals_per_ranking_suf_dif"] = (dif["home_goals_suf_mean"] / dif["home_rank_mean"]) - (dif["away_goals_suf_mean"] / dif["away_rank_mean"])
dif.loc[:, "goals_per_ranking_dif_l5"] = (dif["home_goals_mean_l5"] / dif["home_rank_mean"]) - (dif["away_goals_mean_l5"] / dif["away_rank_mean"])
dif.loc[:, "goals_per_ranking_suf_dif_l5"] = (dif["home_goals_suf_mean_l5"] / dif["home_rank_mean"]) - (dif["away_goals_suf_mean_l5"] / dif["away_rank_mean"])

In [35]:
def create_db(df):
    columns = ["home_team", "away_team", "target", "rank_dif", "home_goals_mean", "home_rank_mean", "away_goals_mean", "away_rank_mean", "home_rank_mean_l5", "away_rank_mean_l5", "home_goals_suf_mean", "away_goals_suf_mean", "home_goals_mean_l5", "away_goals_mean_l5", "home_goals_suf_mean_l5", "away_goals_suf_mean_l5", "home_game_points_rank_mean", "home_game_points_rank_mean_l5", "away_game_points_rank_mean", "away_game_points_rank_mean_l5","is_friendly_0", "is_friendly_1"]
    
    base = df.loc[:, columns]
    base.loc[:, "goals_dif"] = base["home_goals_mean"] - base["away_goals_mean"]
    base.loc[:, "goals_dif_l5"] = base["home_goals_mean_l5"] - base["away_goals_mean_l5"]
    base.loc[:, "goals_suf_dif"] = base["home_goals_suf_mean"] - base["away_goals_suf_mean"]
    base.loc[:, "goals_suf_dif_l5"] = base["home_goals_suf_mean_l5"] - base["away_goals_suf_mean_l5"]
    base.loc[:, "goals_per_ranking_dif"] = (base["home_goals_mean"] / base["home_rank_mean"]) - (base["away_goals_mean"] / base["away_rank_mean"])
    base.loc[:, "dif_rank_agst"] = base["home_rank_mean"] - base["away_rank_mean"]
    base.loc[:, "dif_rank_agst_l5"] = base["home_rank_mean_l5"] - base["away_rank_mean_l5"]
    base.loc[:, "dif_points_rank"] = base["home_game_points_rank_mean"] - base["away_game_points_rank_mean"]
    base.loc[:, "dif_points_rank_l5"] = base["home_game_points_rank_mean_l5"] - base["away_game_points_rank_mean_l5"]
    model_df = base[["home_team", "away_team", "target", "rank_dif", "goals_dif", "goals_dif_l5", "goals_suf_dif", "goals_suf_dif_l5", "goals_per_ranking_dif", "dif_rank_agst", "dif_rank_agst_l5", "dif_points_rank", "dif_points_rank_l5", "is_friendly_0", "is_friendly_1"]]
    return model_df

In [36]:
model_db = create_db(df)
model_db

,home_team,away_team,target,rank_dif,goals_dif,goals_dif_l5,goals_suf_dif,goals_suf_dif_l5,goals_per_ranking_dif,dif_rank_agst,dif_rank_agst_l5,dif_points_rank,dif_points_rank_l5,is_friendly_0,is_friendly_1
0,Albania,Uzbekistan,0,-18,1.00,1.00,-1,-1,0.000000,9.000000,12.000000,0.000000,0.000000,0,1
1,Algeria,Gabon,1,-1,1.00,2.00,0,-1,0.033929,-3.000000,-7.000000,0.000000,0.000000,0,1
2,Angola,Uruguay,1,81,1.00,-2.00,1,-2,0.006192,15.000000,21.000000,0.000000,0.000000,0,1
3,Armenia,Iran,1,31,-1.00,-2.00,0,0,-0.142857,28.000000,-5.000000,0.000000,0.000000,0,1
4,Austria,Switzerland,1,43,-2.00,-1.00,-1,1,-0.074074,4.000000,-13.000000,0.000000,0.000000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,Cameroon,Mali,1,-11,3.00,3.00,1,0,0.053571,-46.000000,-46.000000,0.053571,0.053571,0,1
183,Madagascar,Mauritania,1,5,0.00,2.00,2,1,0.000000,-132.000000,-144.000000,-0.005848,-0.005848,0,1
184,Brazil,Bolivia,0,-59,-0.50,-0.50,-2,2,-0.010753,-41.500000,-41.500000,-0.022661,-0.022661,1,0
185,Turkey,Azerbaijan,0,-81,0.25,0.25,0,1,0.008034,-7.583333,-7.583333,-0.002214,-0.002214,0,1


# MODEL GRADIENT BOOSTING

The gradient boosting model is a machine learning technique used to build predictive models, typically decision trees. It is a type of ensemble method, which means it combines predictions from multiple models to improve overall predictive performance. In gradient boosting, each model is trained to make slightly better predictions than random chance, and the models are then combined in a way that minimizes the overall prediction error. This is done by sequentially training new models to correct the errors of the previous models. The result is a model that makes highly accurate predictions.

In [37]:
X = model_db.iloc[:, 3:]
y = model_db[["target"]]

In [38]:

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=1)
gb = GradientBoostingClassifier(random_state=5)

params = {"learning_rate": [0.01, 0.1, 0.5],
            "min_samples_split": [5, 30],
            "min_samples_leaf": [3, 5],
            "max_depth":[3,5,10],
            "max_features":["sqrt"],
            "n_estimators":[100, 200]
         } 

gb_cv = GridSearchCV(gb, params, cv = 3, n_jobs = -1, verbose = False)

gb_cv.fit(X_train.values, np.ravel(y_train))

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=5),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.5],
                         'max_depth': [3, 5, 10], 'max_features': ['sqrt'],
                         'min_samples_leaf': [3, 5],
                         'min_samples_split': [5, 30],
                         'n_estimators': [100, 200]},
             verbose=False)

In [39]:
gb = gb_cv.best_estimator_

In [40]:
y_pred = gb.predict(X_test.values)

### ACCURACY of the model is equal to  0.66 (66%), So it's a good model

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6578947368421053

In [42]:
def find_features(team_1, team_2):
    rank_dif = team_1[0] - team_2[0]
    goals_dif = team_1[1] - team_2[1]
    goals_dif_l5 = team_1[2] - team_2[2]
    goals_suf_dif = team_1[3] - team_2[3]
    goals_suf_dif_l5 = team_1[4] - team_2[4]
    goals_per_ranking_dif = (team_1[1]/team_1[5]) - (team_2[1]/team_2[5])
    dif_rank_agst = team_1[5] - team_2[5]
    dif_rank_agst_l5 = team_1[6] - team_2[6]
    dif_gp_rank = team_1[7] - team_2[7]
    dif_gp_rank_l5 = team_1[8] - team_2[8]
    
    return [rank_dif, goals_dif, goals_dif_l5, goals_suf_dif, goals_suf_dif_l5, goals_per_ranking_dif, dif_rank_agst, dif_rank_agst_l5, dif_gp_rank, dif_gp_rank_l5, 1, 0]

In [43]:
def find_stats(team_1):
#team_1 = "Qatar"
    past_games = team_stats_raw[(team_stats_raw["team"] == team_1)].sort_values("date")
    last5 = team_stats_raw[(team_stats_raw["team"] == team_1)].sort_values("date").tail(5)

    team_1_rank = past_games["rank"].values[-1]
    team_1_goals = past_games.score.mean()
    team_1_goals_l5 = last5.score.mean()
    team_1_goals_suf = past_games.suf_score.mean()
    team_1_goals_suf_l5 = last5.suf_score.mean()
    team_1_rank_suf = past_games.rank_suf.mean()
    team_1_rank_suf_l5 = last5.rank_suf.mean()
    team_1_gp_rank = past_games.points_by_rank.mean()
    team_1_gp_rank_l5 = last5.points_by_rank.mean()

    return [team_1_rank, team_1_goals, team_1_goals_l5, team_1_goals_suf, team_1_goals_suf_l5, team_1_rank_suf, team_1_rank_suf_l5, team_1_gp_rank, team_1_gp_rank_l5]

In [44]:
matches = [('Argentina','Croatia'),('France','Morocco')]
final=[]
for teams in matches:
    draw = False
    team_1 = find_stats(teams[0])
    team_2 = find_stats(teams[1])

    

    features_g1 = find_features(team_1, team_2)
    features_g2 = find_features(team_2, team_1)

    probs_g1 = gb.predict_proba([features_g1])
    probs_g2 = gb.predict_proba([features_g2])
    
    team_1_prob_g1 = probs_g1[0][0]
    team_1_prob_g2 = probs_g2[0][1]
    team_2_prob_g1 = probs_g1[0][1]
    team_2_prob_g2 = probs_g2[0][0]

    team_1_prob = (probs_g1[0][0] + probs_g2[0][1])/2
    team_2_prob = (probs_g2[0][0] + probs_g1[0][1])/2
    
    
                
    if team_1_prob > team_2_prob:
        winner = teams[0]
        winner_proba = team_1_prob
        final.append(winner)
        print('The Winner is '+teams[0]+' with probability equal to '+str(winner_proba)+', and it has qualified to THE FINAL')
        print("                                                                                                            ")        
    elif team_2_prob > team_1_prob:  
        winner = teams[1]
        winner_proba = team_2_prob
        final.append(winner)
        print('The Winner is '+teams[1]+' with probability equal to '+str(winner_proba)+', and it has qualified to THE FINAL')
        
final    

The Winner is Argentina with probability equal to 0.7744048867916962, and it has qualified to THE FINAL
                                                                                                            
The Winner is France with probability equal to 0.6891732694233674, and it has qualified to THE FINAL
                                                                                                            


['Argentina', 'France']

In [45]:
print("---------------------------THE WORLD CUP WINNER-------------------------------")

team_1 = find_stats(final[0])
team_2 = find_stats(final[1])

    

features_g1 = find_features(team_1, team_2)
features_g2 = find_features(team_2, team_1)

probs_g1 = gb.predict_proba([features_g1])
probs_g2 = gb.predict_proba([features_g2])
    
team_1_prob_g1 = probs_g1[0][0]
team_1_prob_g2 = probs_g2[0][1]
team_2_prob_g1 = probs_g1[0][1]
team_2_prob_g2 = probs_g2[0][0]

team_1_prob = (probs_g1[0][0] + probs_g2[0][1])/2
team_2_prob = (probs_g2[0][0] + probs_g1[0][1])/2
    
    
                
if team_1_prob > team_2_prob:
        winner = final[0]
        winner_proba = team_1_prob
        
        print('The World Cup Winner is '+final[0]+' with probability equal to '+str(winner_proba))
        print("                                                                                                            ")        
elif team_2_prob > team_1_prob:  
        winner = final[1]
        winner_proba = team_2_prob
        
        print('The World Cup Winner is '+final[1]+' with probability equal to '+str(winner_proba))

---------------------------THE WORLD CUP WINNER-------------------------------
The World Cup Winner is Argentina with probability equal to 0.6939794779339159
                                                                                                            
